# Анализ эксперимента с NeuralControllerEnv

In [ ]:
import matplotlib.pyplot as plt

from pathlib import Path
from nn_laser_stabilizer.config.config import load_config

EXPERIMENT_DATE_TIME = "YYYY-MM-DD_HH-MM-SS"
EXPERIMENT_NAME = "neural_controller"

EXPERIMENT_DIR_PATH = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE_TIME}")

CONFIG_PATH = EXPERIMENT_DIR_PATH / "config.yaml"
config = load_config(CONFIG_PATH)
print(f"Эксперимент: {config.experiment_name}")

## Анализ логов окружения

In [ ]:
import re
import numpy as np
import pandas as pd

def parse_env_logs(file_path: str | Path) -> pd.DataFrame:
    step_pattern = re.compile(
        r"\[ENV\]\s+"
        r"step:\s+"
        r"step=(?P<step>\d+)\s+"
        r"process_variable=(?P<process_variable>\d+)\s+"
        r"setpoint=(?P<setpoint>\d+)\s+"
        r"error=(?P<error>-?\d+\.\d+)\s+"
        r"action=(?P<action>-?\d+\.\d+)\s+"
        r"control_output=(?P<control_output>\d+)\s+"
        r"reward=(?P<reward>-?\d+\.\d+)"
    )
    
    reset_pattern = re.compile(
        r"\[ENV\]\s+"
        r"reset:\s+"
        r"process_variable=(?P<process_variable>\d+)\s+"
        r"setpoint=(?P<setpoint>\d+)\s+"
        r"error=(?P<error>-?\d+\.\d+)\s+"
        r"control_output=(?P<control_output>\d+)"
    )
    
    env_rows = []
    current_step = 0
    
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            m = reset_pattern.match(line)
            if m:
                env_rows.append({
                    'Step': current_step,
                    'Type': 'reset',
                    'Process variable': int(m.group('process_variable')),
                    'Setpoint': int(m.group('setpoint')),
                    'Error': float(m.group('error')),
                    'Action': np.nan,
                    'Control output': int(m.group('control_output')),
                    'Reward': np.nan,
                })
                continue
            
            m = step_pattern.match(line)
            if m:
                current_step = int(m.group('step'))
                env_rows.append({
                    'Step': current_step,
                    'Type': 'step',
                    'Process variable': int(m.group('process_variable')),
                    'Setpoint': int(m.group('setpoint')),
                    'Error': float(m.group('error')),
                    'Action': float(m.group('action')),
                    'Control output': int(m.group('control_output')),
                    'Reward': float(m.group('reward')),
                })
                continue
    
    return pd.DataFrame(env_rows)

In [ ]:
ENV_LOG_PATH = EXPERIMENT_DIR_PATH / "env_logs" / "env.log"
env_df = parse_env_logs(ENV_LOG_PATH)
print(f"Загружено {len(env_df)} записей из логов окружения")

In [ ]:
step_df = env_df[env_df['Type'] == 'step'].copy()
exploration_steps = config.exploration.steps

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(step_df['Step'], step_df['Error'], alpha=0.8, linewidth=0.8, label='Error')

plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2, 
            label=f'Switch to NN (step {exploration_steps})')

plt.title(f'Error over Steps')
plt.xlabel('Step')
plt.ylabel('Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(step_df['Step'], step_df['Reward'], alpha=0.8, linewidth=0.8, label='Reward')

plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2, 
            label=f'Switch to NN (step {exploration_steps})')

plt.title('Reward over Steps')
plt.xlabel('Step')
plt.ylabel('Reward')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(step_df['Step'], step_df['Process variable'], 'b-', alpha=0.7, linewidth=0.8, label='Process Variable')
plt.plot(step_df['Step'], step_df['Setpoint'], color='r', linestyle='--', linewidth=1.5, label='Setpoint')

plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2, 
            label=f'Switch to NN (step {exploration_steps})')

plt.title('Process Variable over Steps')
plt.xlabel('Step')
plt.ylabel('Value')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(step_df['Step'], step_df['Control output'], 'g-', alpha=0.7, linewidth=0.8, label='Control Output')
plt.xlabel('Step', fontsize=12)
plt.ylabel('Control Output', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(loc='best')

plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2, 
            label=f'Switch to NN (step {exploration_steps})')

plt.title('Control Output over Steps')
plt.tight_layout()
plt.show()

## Анализ процесса обучения

In [ ]:
def parse_train_logs(file_path):
    step_pattern = re.compile(
        r"\[TRAIN\]\s+"
        r"step:\s+"
        r"(actor_loss=(?P<actor_loss>-?\d+\.\d+)\s+)?"
        r"buffer_size=(?P<buffer_size>\d+)\s+"
        r"loss_q1=(?P<loss_q1>-?\d+\.\d+)\s+"
        r"loss_q2=(?P<loss_q2>-?\d+\.\d+)\s+"
        r"step=(?P<step>\d+)\s+"
        r"time=(?P<time>-?\d+\.\d+)"
    )
    
    rows = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            match = step_pattern.match(line)
            if match:
                actor_loss = match.group('actor_loss')
                rows.append({
                    'step': int(match.group('step')),
                    'loss_q1': float(match.group('loss_q1')),
                    'loss_q2': float(match.group('loss_q2')),
                    'actor_loss': float(actor_loss) if actor_loss else np.nan,
                    'buffer_size': int(match.group('buffer_size'))
                })
    
    return pd.DataFrame(rows)

In [ ]:
TRAIN_LOG_PATH = EXPERIMENT_DIR_PATH / "train_logs" / "train.log"
loss_df = parse_train_logs(TRAIN_LOG_PATH)
print(f"Загружено {len(loss_df)} записей из логов обучения")
print(f"Диапазон шагов обучения: {loss_df['step'].min()} - {loss_df['step'].max()}")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

axes[0].plot(loss_df['step'], loss_df['loss_q1'], 'b-', alpha=0.7, label='Q1 Loss')
axes[0].set_title('Q1 Loss')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(loss_df['step'], loss_df['loss_q2'], 'g-', alpha=0.7, label='Q2 Loss')
axes[1].set_title('Q2 Loss')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(loss_df['step'], loss_df['loss_q1'] + loss_df['loss_q2'], 'r--', alpha=0.7, label='Sum (Q1 + Q2)')
axes[2].set_title('Sum (Q1 + Q2)')
axes[2].set_xlabel('Step')
axes[2].set_ylabel('Loss')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
actor_loss_df = loss_df[loss_df['actor_loss'].notna()]

plt.figure(figsize=(12, 5))

plt.plot(actor_loss_df['step'], actor_loss_df['actor_loss'], 'r-', alpha=0.7)
plt.title('Actor Loss')

plt.xlabel('Step')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(loss_df['step'], loss_df['buffer_size'], 'm-', alpha=0.7)
plt.title('Buffer Size')
plt.xlabel('Step')
plt.ylabel('Size')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()